In [1]:
import pandas as pd
import numpy as np

In [2]:
date_1 = pd.read_csv("date_1.csv")
date_2 = pd.read_csv("date_2.csv")
date_3 = pd.read_csv("date_3.csv")
date_4 = pd.read_csv("date_4.csv")
date_5 = pd.read_csv("date_5.csv")

full_date_df = pd.concat([date_1,date_2,date_3,date_4,date_5])

len(full_date_df)

263650

In [3]:
full_date_df.head()

,created_utc
0,1201236548
1,1201236599
2,1201236626
3,1201236662
4,1201236696


In [4]:
title_1 = pd.read_csv("title_1.csv")
title_2 = pd.read_csv("title_2.csv")
title_3 = pd.read_csv("title_3.csv")
title_4 = pd.read_csv("title_4.csv")
title_5 = pd.read_csv("title_5.csv")

full_title_df = pd.concat([title_1,title_2,title_3,title_4,title_5])

len(full_title_df)

314924

In [5]:
full_title_df.head()

,title
0,The Best Economics Blog Going
1,Meta: Seed The Economics blog with direct link...
2,Meta: Seed The Economics subreddit with direct...
3,Calculated Risk
4,Chris Blattman's Blog


In [6]:
full_df = pd.concat([full_date_df.reset_index(), full_title_df.reset_index()], axis=1)

In [7]:
full_df_clean = full_df.dropna()

In [8]:
print(full_df_clean.shape)

(263650, 4)


In [9]:
full_df_clean.head()

,index,created_utc,index,title
0,0.0,1.201237e+09,0,The Best Economics Blog Going
1,1.0,1.201237e+09,1,Meta: Seed The Economics blog with direct link...
2,2.0,1.201237e+09,2,Meta: Seed The Economics subreddit with direct...
3,3.0,1.201237e+09,3,Calculated Risk
4,4.0,1.201237e+09,4,Chris Blattman's Blog


In [10]:
full_df_clean = full_df_clean[['created_utc','title']]

In [11]:
import datetime

In [12]:
full_df_clean['date'] = pd.to_datetime(full_df_clean['created_utc'], unit = "s")
full_df_clean.head()

,created_utc,title,date
0,1.201237e+09,The Best Economics Blog Going,2008-01-25 04:49:08
1,1.201237e+09,Meta: Seed The Economics blog with direct link...,2008-01-25 04:49:59
2,1.201237e+09,Meta: Seed The Economics subreddit with direct...,2008-01-25 04:50:26
3,1.201237e+09,Calculated Risk,2008-01-25 04:51:02
4,1.201237e+09,Chris Blattman's Blog,2008-01-25 04:51:36


In [13]:
full_df_clean['date'] = full_df_clean['date'].dt.date
full_df_clean.head()

,created_utc,title,date
0,1.201237e+09,The Best Economics Blog Going,2008-01-25
1,1.201237e+09,Meta: Seed The Economics blog with direct link...,2008-01-25
2,1.201237e+09,Meta: Seed The Economics subreddit with direct...,2008-01-25
3,1.201237e+09,Calculated Risk,2008-01-25
4,1.201237e+09,Chris Blattman's Blog,2008-01-25


In [14]:
full_df_clean = full_df_clean[['date', 'title']]
full_df_clean.head()

,date,title
0,2008-01-25,The Best Economics Blog Going
1,2008-01-25,Meta: Seed The Economics blog with direct link...
2,2008-01-25,Meta: Seed The Economics subreddit with direct...
3,2008-01-25,Calculated Risk
4,2008-01-25,Chris Blattman's Blog


In [15]:
import nltk

In [16]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [17]:
# Index of the row you want to analyze (e.g., row 1)
#row_index = 1

analyzer = SentimentIntensityAnalyzer()

In [18]:
scores = []
for i in range(0,len(full_df_clean['title'])):
    row_index = i
    text_to_analyse = full_df_clean.loc[row_index,'title']
    sentiment_scores = analyzer.polarity_scores(text_to_analyse)
    compound_score = sentiment_scores['compound']
    scores.append(compound_score)

In [19]:
len(scores)

263650

In [20]:
full_df_clean['scores'] = scores
full_df_clean.head()

,date,title,scores
0,2008-01-25,The Best Economics Blog Going,0.6369
1,2008-01-25,Meta: Seed The Economics blog with direct link...,0.4588
2,2008-01-25,Meta: Seed The Economics subreddit with direct...,0.4588
3,2008-01-25,Calculated Risk,-0.2732
4,2008-01-25,Chris Blattman's Blog,0.0000


In [21]:
full_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 263650 entries, 0 to 263649
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   date    263650 non-null  object 
 1   title   263650 non-null  object 
 2   scores  263650 non-null  float64
dtypes: float64(1), object(2)
memory usage: 16.1+ MB


In [22]:
full_df_clean['date'] = pd.to_datetime(full_df_clean['date'])

In [23]:
full_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 263650 entries, 0 to 263649
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   date    263650 non-null  datetime64[ns]
 1   title   263650 non-null  object        
 2   scores  263650 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 16.1+ MB


In [24]:
full_df_clean.head()

,date,title,scores
0,2008-01-25,The Best Economics Blog Going,0.6369
1,2008-01-25,Meta: Seed The Economics blog with direct link...,0.4588
2,2008-01-25,Meta: Seed The Economics subreddit with direct...,0.4588
3,2008-01-25,Calculated Risk,-0.2732
4,2008-01-25,Chris Blattman's Blog,0.0000


In [25]:
sentiment_df = full_df_clean.groupby('date')['scores'].mean().reset_index()

In [26]:
sentiment_df.head()

,date,scores
0,2008-01-25,0.084243
1,2008-01-26,0.003900
2,2008-01-27,0.075342
3,2008-01-28,-0.027604
4,2008-01-29,0.030093


In [27]:
sentiment_df.tail()

,date,scores
4751,2022-12-27,0.083791
4752,2022-12-28,-0.019761
4753,2022-12-29,0.011074
4754,2022-12-30,0.020986
4755,2022-12-31,-0.193041


In [28]:
for i in range(0,len(sentiment_df['scores'])):
    if sentiment_df.loc[i,'scores'] >= 0.05:
        sentiment_df.loc[i,'sentiment'] = 'Positive'
    elif sentiment_df.loc[i,'scores'] <= -0.05:
        sentiment_df.loc[i,'sentiment'] = 'Negative'
    else:
        sentiment_df.loc[i,'sentiment'] = 'Neutral'

In [29]:
sentiment_df.head()

,date,scores,sentiment
0,2008-01-25,0.084243,Positive
1,2008-01-26,0.003900,Neutral
2,2008-01-27,0.075342,Positive
3,2008-01-28,-0.027604,Neutral
4,2008-01-29,0.030093,Neutral


In [30]:
sp_500 = pd.read_excel("SP_500_prices.xlsx")
sp_500.head()

,Dates,PX_LAST
0,2008-01-25,1330.61
1,2008-01-28,1353.97
2,2008-01-29,1362.30
3,2008-01-30,1355.81
4,2008-01-31,1378.55


In [31]:
sp_sentiment = pd.merge(sp_500, sentiment_df, left_on = "Dates", right_on = "date", how = "left")
sp_sentiment.head()

,Dates,PX_LAST,date,scores,sentiment
0,2008-01-25,1330.61,2008-01-25,0.084243,Positive
1,2008-01-28,1353.97,2008-01-28,-0.027604,Neutral
2,2008-01-29,1362.30,2008-01-29,0.030093,Neutral
3,2008-01-30,1355.81,2008-01-30,-0.115100,Negative
4,2008-01-31,1378.55,2008-01-31,-0.014772,Neutral


In [32]:
for i in range(0,len(sp_sentiment.scores) - 1):
    if sp_sentiment.loc[i+1,'PX_LAST'] > sp_sentiment.loc[i,'PX_LAST']:
        sp_sentiment.loc[i+1,'Direction'] = 'Up'
    else:
        sp_sentiment.loc[i+1,'Direction'] = 'Down'

In [33]:
sp_sentiment.head()

,Dates,PX_LAST,date,scores,sentiment,Direction
0,2008-01-25,1330.61,2008-01-25,0.084243,Positive,NaN
1,2008-01-28,1353.97,2008-01-28,-0.027604,Neutral,Up
2,2008-01-29,1362.30,2008-01-29,0.030093,Neutral,Up
3,2008-01-30,1355.81,2008-01-30,-0.115100,Negative,Down
4,2008-01-31,1378.55,2008-01-31,-0.014772,Neutral,Up


In [34]:
sp_sentiment.tail()

,Dates,PX_LAST,date,scores,sentiment,Direction
3756,2022-12-23,3844.82,2022-12-23,-0.131208,Negative,Up
3757,2022-12-27,3829.25,2022-12-27,0.083791,Positive,Down
3758,2022-12-28,3783.22,2022-12-28,-0.019761,Neutral,Down
3759,2022-12-29,3849.28,2022-12-29,0.011074,Neutral,Up
3760,2022-12-30,3839.50,2022-12-30,0.020986,Neutral,Down


In [35]:
sp_sentiment['sentiment_lead'] = sp_sentiment['sentiment'].shift(periods=1, fill_value=None)

In [36]:
sp_sentiment.head()

,Dates,PX_LAST,date,scores,sentiment,Direction,sentiment_lead
0,2008-01-25,1330.61,2008-01-25,0.084243,Positive,NaN,NaN
1,2008-01-28,1353.97,2008-01-28,-0.027604,Neutral,Up,Positive
2,2008-01-29,1362.30,2008-01-29,0.030093,Neutral,Up,Neutral
3,2008-01-30,1355.81,2008-01-30,-0.115100,Negative,Down,Neutral
4,2008-01-31,1378.55,2008-01-31,-0.014772,Neutral,Up,Negative


In [37]:
sp_sent_df_clean = sp_sentiment.dropna()
sp_sent_df_clean.head()

,Dates,PX_LAST,date,scores,sentiment,Direction,sentiment_lead
1,2008-01-28,1353.97,2008-01-28,-0.027604,Neutral,Up,Positive
2,2008-01-29,1362.30,2008-01-29,0.030093,Neutral,Up,Neutral
3,2008-01-30,1355.81,2008-01-30,-0.115100,Negative,Down,Neutral
4,2008-01-31,1378.55,2008-01-31,-0.014772,Neutral,Up,Negative
5,2008-02-01,1395.42,2008-02-01,-0.237369,Negative,Up,Neutral


In [38]:
sp_sent_df_clean.head()

,Dates,PX_LAST,date,scores,sentiment,Direction,sentiment_lead
1,2008-01-28,1353.97,2008-01-28,-0.027604,Neutral,Up,Positive
2,2008-01-29,1362.30,2008-01-29,0.030093,Neutral,Up,Neutral
3,2008-01-30,1355.81,2008-01-30,-0.115100,Negative,Down,Neutral
4,2008-01-31,1378.55,2008-01-31,-0.014772,Neutral,Up,Negative
5,2008-02-01,1395.42,2008-02-01,-0.237369,Negative,Up,Neutral


In [39]:
sp_sent_df_clean['Direction_lag'] = sp_sent_df_clean['Direction'].shift(periods=-1, fill_value=None)
sp_sent_df_clean.head()

/var/folders/4y/k0ycwfqd4ns75ty_9446wbp00000gn/T/ipykernel_40839/3257235865.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp_sent_df_clean['Direction_lag'] = sp_sent_df_clean['Direction'].shift(periods=-1, fill_value=None)


,Dates,PX_LAST,date,scores,sentiment,Direction,sentiment_lead,Direction_lag
1,2008-01-28,1353.97,2008-01-28,-0.027604,Neutral,Up,Positive,Up
2,2008-01-29,1362.30,2008-01-29,0.030093,Neutral,Up,Neutral,Down
3,2008-01-30,1355.81,2008-01-30,-0.115100,Negative,Down,Neutral,Up
4,2008-01-31,1378.55,2008-01-31,-0.014772,Neutral,Up,Negative,Up
5,2008-02-01,1395.42,2008-02-01,-0.237369,Negative,Up,Neutral,Down


In [40]:
sp_sent_df_clean = sp_sent_df_clean.reset_index()
sp_sent_df_clean.head()

,index,Dates,PX_LAST,date,scores,sentiment,Direction,sentiment_lead,Direction_lag
0,1,2008-01-28,1353.97,2008-01-28,-0.027604,Neutral,Up,Positive,Up
1,2,2008-01-29,1362.30,2008-01-29,0.030093,Neutral,Up,Neutral,Down
2,3,2008-01-30,1355.81,2008-01-30,-0.115100,Negative,Down,Neutral,Up
3,4,2008-01-31,1378.55,2008-01-31,-0.014772,Neutral,Up,Negative,Up
4,5,2008-02-01,1395.42,2008-02-01,-0.237369,Negative,Up,Neutral,Down


In [41]:
sentiment_lag = []

for i in range(0,len(sp_sent_df_clean.scores)):
    if sp_sent_df_clean.loc[i,'sentiment'] == 'Negative':
        sentiment_lag.append(0)
    elif sp_sent_df_clean.loc[i,'sentiment'] == 'Neutral':
        sentiment_lag.append(1)
    elif sp_sent_df_clean.loc[i,'sentiment'] == 'Positive':
        sentiment_lag.append(2)

In [42]:
sentiment_lead = []

for i in range(0,len(sp_sent_df_clean.scores)):
    if sp_sent_df_clean.loc[i,'sentiment_lead'] == 'Negative':
        sentiment_lead.append(0)
    elif sp_sent_df_clean.loc[i,'sentiment_lead'] == 'Neutral':
        sentiment_lead.append(1)
    elif sp_sent_df_clean.loc[i,'sentiment_lead'] == 'Positive':
        sentiment_lead.append(2)

In [43]:
direction = []

for i in range(0,len(sp_sent_df_clean.scores)):
    if sp_sent_df_clean.loc[i,'Direction'] == 'Up':
        direction.append(1)
    else:
        direction.append(0)

In [44]:
direction_lag = []

for i in range(0,len(sp_sent_df_clean.scores)):
    if sp_sent_df_clean.loc[i,'Direction_lag'] == 'Up':
        direction_lag.append(1)
    else:
        direction_lag.append(0)

In [46]:
data = {'date': sp_sent_df_clean.Dates, 'sentiment_lag': sentiment_lag, 'sentiment_lead': sentiment_lead, 
        'direction_lag': direction_lag, 'direction': direction}
df = pd.DataFrame(data)
df.head()

,date,sentiment_lag,sentiment_lead,direction_lag,direction
0,2008-01-28,1,2,1,1
1,2008-01-29,1,1,0,1
2,2008-01-30,0,1,1,0
3,2008-01-31,1,0,1,1
4,2008-02-01,0,1,0,1


In [47]:
#import tensorflow as tf
#from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [55]:
X = np.array(df[['sentiment_lag', 'sentiment_lead', 'direction_lag']])
y = np.array(df.direction)

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [57]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### ANN

In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [ ]:
def create_model(optimizer='adam', hidden_units_1=12, hidden_units_2 = 8, activation='relu'):
    model = Sequential()
    model.add(Dense(hidden_units_1, activation=activation, input_dim=4))
    model.add(Dense(hidden_units_2, activation=activation, input_dim=4))
    model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [ ]:
model = keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, verbose=0)

In [ ]:
param_grid = {
    'batch_size': [10, 20],
    'epochs': [100, 150],
    'hidden_units_1': [12, 24, 36],
    'hidden_units_2': [4, 8, 12],
    'activation': ['relu', 'sigmoid'],
    'optimizers': ['adam', 'sgd', 'rmsprop']
}

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)

In [ ]:
# Perform the grid search
grid_result = grid.fit(X_train, y_train)

In [ ]:
# Get the best hyperparameters
best_params = grid_result.best_params_
print(f'Best Hyperparameters: {best_params}')

In [ ]:
best_model = grid_result.best_estimator_
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy on Test Set: {accuracy * 100:.2f}%')